# Spark streaming

In [2]:
from pyspark.sql import SparkSession

spark = (
    SparkSession
    .builder
    .config("spark.sql.streaming.schemaInference", True)
    .getOrCreate()
)

stream = (
    spark
    .readStream
    .format("ws")
    .option("schema", "ticker")
    .load()
)

query = (
    stream
    .select("side", "product_id", "last_size", "best_bid", "best_ask", "time")
    .writeStream
    .format("console")
    .outputMode("append")
    .option("truncate", "false")
    .start()
)

query.awaitTermination(10)
query.stop()
stream.printSchema()

root
 |-- type: string (nullable = false)
 |-- trade_id: long (nullable = false)
 |-- sequence: long (nullable = false)
 |-- time: timestamp (nullable = false)
 |-- product_id: string (nullable = false)
 |-- price: double (nullable = false)
 |-- side: string (nullable = false)
 |-- last_size: double (nullable = false)
 |-- best_bid: double (nullable = false)
 |-- best_ask: double (nullable = false)



In [ ]:
# panic button - press only if you messed up opening new websocket and lost reference to it

query.stop()
spark.stop()

## CoinBase stream analysis

Write a query to print price (`price` parameter) in moving time window, moving along transaction time (`time` column), grouping by exchange relation (from which currency to which one, `product_id` column).

In [4]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = (
    SparkSession
    .builder
    .config("spark.sql.streaming.schemaInference", True)
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", True)
    .getOrCreate()
)

stream = (
    spark
    .readStream
    .format("ws")
    .option("schema", "ticker")
    .load()
)


query = (
    stream
    .select("product_id", "price", "time")
    .groupBy(F.window("time", "2 minutes", "1 minutes"), "product_id").mean("price")
    .orderBy("window")
)

query = (
    query
    .writeStream
    .format("console")
    .outputMode("complete")
    .option("truncate", "false")
    .start()
)

query.awaitTermination(10)
query.stop()

21/11/28 18:09:55 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-ed8821b1-0e58-48a7-ab6d-cc8964a6a0a1. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/11/28 18:09:55 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----------+----------+
|window|product_id|avg(price)|
+------+----------+----------+
+------+----------+----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+----------+-----------------+
|window                                    |product_id|avg(price)       |
+------------------------------------------+----------+-----------------+
|{2021-11-28 18:08:00, 2021-11-28 18:10:00}|ETH-BTC   |0.07494          |
|{2021-11-28 18:08:00, 2021-11-28 18:10:00}|ETH-USD   |4046.056666666667|
|{2021-11-28 18:09:00, 2021-11-28 18:11:00}|ETH-BTC   |0.07494          |
|{2021-11-28 18:09:00, 2021-11-28 18:11:00}|ETH-USD   |4046.056666666667|
+------------------------------------------+----------+-----------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+------------------------------------------+----------+------------------+
|window                                    |product_id|avg(price)        |
+------------------------------------------+----------+------------------+
|{2021-11-28 18:08:00, 2021-11-28 18:10:00}|ETH-BTC   |0.07494           |
|{2021-11-28 18:08:00, 2021-11-28 18:10:00}|ETH-USD   |4046.337777777778 |
|{2021-11-28 18:09:00, 2021-11-28 18:11:00}|ETH-BTC   |0.074955          |
|{2021-11-28 18:09:00, 2021-11-28 18:11:00}|ETH-USD   |4046.619565217392 |
|{2021-11-28 18:10:00, 2021-11-28 18:12:00}|ETH-BTC   |0.07497           |
|{2021-11-28 18:10:00, 2021-11-28 18:12:00}|ETH-USD   |4047.6339999999996|
+------------------------------------------+----------+------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+----------+------------------+
|window                                    |product_id|avg(price)        |
+------------------------------------------+----------+------------------+
|{2021-11-28 18:08:00, 2021-11-28 18:10:00}|ETH-BTC   |0.07494           |
|{2021-11-28 18:08:00, 2021-11-28 18:10:00}|ETH-USD   |4046.337777777778 |
|{2021-11-28 18:09:00, 2021-11-28 18:11:00}|ETH-BTC   |0.074955          |
|{2021-11-28 18:09:00, 2021-11-28 18:11:00}|ETH-USD   |4046.6900000000005|
|{2021-11-28 18:10:00, 2021-11-28 18:12:00}|ETH-BTC   |0.07497           |
|{2021-11-28 18:10:00, 2021-11-28 18:12:00}|ETH-USD   |4047.2663636363636|
+------------------------------------------+----------+------------------+



21/11/28 18:10:05 WARN Shell: Interrupted while joining on: Thread[Thread-181350,5,]
java.lang.InterruptedException
	at java.base/java.lang.Object.wait(Native Method)
	at java.base/java.lang.Thread.join(Thread.java:1300)
	at java.base/java.lang.Thread.join(Thread.java:1375)
	at org.apache.hadoop.util.Shell.joinThread(Shell.java:1043)
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1003)
	at org.apache.hadoop.util.Shell.run(Shell.java:901)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1213)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1307)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1289)
	at org.apache.hadoop.fs.FileUtil.readLink(FileUtil.java:211)
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileLinkStatusInternal(RawLocalFileSystem.java:1113)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:1102)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatus(RawLocalFileS

## Watermarking and late data

Modify previous query to show watermarks mechanism and late data handling. It should be visible in the output that relevant rows in the output table are being updated (update mode), in particular update of previous time windows after arrival of late data. Provide example output and description.

Use script in `/mock` directory to generate data in batches every 10 seconds:
- fist series: messages with timestamps 0s, 14s, 7s
- second series: messages with timestamps 15s, 8s, 21s
- third series: messages with timestamps 4s, 17s

In [2]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F


spark = (
    SparkSession
    .builder
    .config("spark.sql.streaming.schemaInference", True)
    .getOrCreate()
)

stream = (
    spark
    .readStream
    .format("ws")
    .option("schema", "ticker")
    .option("url", "ws://mock:8025")
    .load()
)


query = (
    stream
    .select("product_id", "price", "time")
    .withWatermark("time", "30 seconds")
    .groupBy(F.window("time", "10 seconds", "5 seconds"), "product_id").mean("price")
)

query = (
    query
    .writeStream
    .format("console")
    .outputMode("update")
    .option("truncate", "false")
    .start()
)

query.awaitTermination(40)
query.stop()


21/11/28 17:45:31 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-b6a1f404-f1c6-4b73-98bd-7ce56323471d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/11/28 17:45:31 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+------+----------+----------+
|window|product_id|avg(price)|
+------+----------+----------+
+------+----------+----------+



-------------------------------------------
Batch: 1
-------------------------------------------
+------------------------------------------+----------+-----------------+
|window                                    |product_id|avg(price)       |
+------------------------------------------+----------+-----------------+
|{2021-11-01 00:00:05, 2021-11-01 00:00:15}|ETH-USD   |339.4526661777917|
|{2021-11-01 00:00:00, 2021-11-01 00:00:10}|ETH-USD   |504.2822544704219|
|{2021-11-01 00:00:10, 2021-11-01 00:00:20}|ETH-USD   |324.4329982157316|
|{2021-10-31 23:59:55, 2021-11-01 00:00:05}|ETH-USD   |654.092174800992 |
+------------------------------------------+----------+-----------------+



-------------------------------------------
Batch: 2
-------------------------------------------
+------+----------+----------+
|window|product_id|avg(price)|
+------+----------+----------+
+------+----------+----------+



-------------------------------------------
Batch: 3
-------------------------------------------
+------------------------------------------+----------+------------------+
|window                                    |product_id|avg(price)        |
+------------------------------------------+----------+------------------+
|{2021-11-01 00:00:05, 2021-11-01 00:00:15}|ETH-USD   |467.3129151426692 |
|{2021-11-01 00:00:20, 2021-11-01 00:00:30}|ETH-USD   |328.9374339872395 |
|{2021-11-01 00:00:00, 2021-11-01 00:00:10}|ETH-USD   |577.1993073377561 |
|{2021-11-01 00:00:10, 2021-11-01 00:00:20}|ETH-USD   |163.2719779271402 |
|{2021-11-01 00:00:15, 2021-11-01 00:00:25}|ETH-USD   |165.52419581289416|
+------------------------------------------+----------+------------------+



-------------------------------------------
Batch: 4
-------------------------------------------
+------+----------+----------+
|window|product_id|avg(price)|
+------+----------+----------+
+------+----------+----------+



-------------------------------------------
Batch: 5
-------------------------------------------
+------------------------------------------+----------+------------------+
|window                                    |product_id|avg(price)        |
+------------------------------------------+----------+------------------+
|{2021-11-01 00:00:00, 2021-11-01 00:00:10}|ETH-USD   |643.5735488840181 |
|{2021-11-01 00:00:10, 2021-11-01 00:00:20}|ETH-USD   |190.8263802410136 |
|{2021-10-31 23:59:55, 2021-11-01 00:00:05}|ETH-USD   |748.3942241618983 |
|{2021-11-01 00:00:15, 2021-11-01 00:00:25}|ETH-USD   |192.32785883151624|
+------------------------------------------+----------+------------------+



### Batch 1

```
2021-10-31 23:59:55, 2021-11-01 00:00:05 -> 0s timestamp update
2021-11-01 00:00:00, 2021-11-01 00:00:10 -> 0s, 7s timestamps updates
2021-11-01 00:00:05, 2021-11-01 00:00:15 -> 7s, 14s timestamps updates
2021-11-01 00:00:10, 2021-11-01 00:00:20 -> 14s timestamp update
```

### Batch 2

No updates

### Batch 3

```
2021-10-31 23:59:55, 2021-11-01 00:00:05 -> no updates
2021-11-01 00:00:00, 2021-11-01 00:00:10 -> 8s timestamp update
2021-11-01 00:00:05, 2021-11-01 00:00:15 -> 8s timestamp update
2021-11-01 00:00:10, 2021-11-01 00:00:20 -> 15s timestamp update
2021-11-01 00:00:15, 2021-11-01 00:00:25 -> 15s, 21s timestamps updates (new window)
2021-11-01 00:00:20, 2021-11-01 00:00:30 -> 21s timestamp update (new window)
```

### Batch 4

No updates

### Batch 5

```
2021-10-31 23:59:55, 2021-11-01 00:00:05 -> 4s timestamp update
2021-11-01 00:00:00, 2021-11-01 00:00:10 -> 4s timestamp update
2021-11-01 00:00:05, 2021-11-01 00:00:15 -> no updates
2021-11-01 00:00:10, 2021-11-01 00:00:20 -> 17s timestamp update
2021-11-01 00:00:15, 2021-11-01 00:00:25 -> 17s timestamp update
2021-11-01 00:00:20, 2021-11-01 00:00:30 -> no updates
```

30s watermark is enough to make all updates apply.

## Stream join

Divide CoinBase data from `ticker` channel into 2 streams, using subcription filtering: one stream for `size="sell"`, second for `side="buy"`. Then create a query to join those streams and prints transactions for given `product_id` that happened one after another during 1s.

In [3]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

spark = (
    SparkSession
    .builder
    .config("spark.sql.streaming.schemaInference", True)
    .config("spark.sql.streaming.forceDeleteTempCheckpointLocation", True)
    .getOrCreate()
)

stream = (
    spark
    .readStream
    .format("ws")
    .option("schema", "ticker")
    .load()
)


sell = (
    stream
    .select("side", "product_id", "price", "time")
    .filter(stream.side == "sell")
    .withColumnRenamed("side", "sell_side")
    .withColumnRenamed("product_id", "sell_product_id")
    .withColumnRenamed("price", "sell_price")
    .withColumnRenamed("time", "sell_time")
)


buy = (
    stream
    .select("side", "product_id", "price", "time")
    .filter(stream.side == "buy")
    .withColumnRenamed("side", "buy_side")
    .withColumnRenamed("product_id", "buy_product_id")
    .withColumnRenamed("price", "buy_price")
    .withColumnRenamed("time", "buy_time")
)


joined = (
    sell.join(
        buy,
        F.expr(
            """
            buy_product_id = sell_product_id AND
            buy_time >= sell_time AND
            buy_time <= sell_time + interval 1 seconds
            """
        )
    )
)

query = (
    joined
    .writeStream
    .format("console")
    .outputMode("append")
    .option("truncate", "false")
    .start()
)

query.awaitTermination(60)
query.stop()

21/11/28 18:04:58 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-8a3e389c-a037-4e6a-ae6c-76e34d83cb0d. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
21/11/28 18:04:58 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+---------+---------------+----------+---------+--------+--------------+---------+--------+
|sell_side|sell_product_id|sell_price|sell_time|buy_side|buy_product_id|buy_price|buy_time|
+---------+---------------+----------+---------+--------+--------------+---------+--------+
+---------+---------------+----------+---------+--------+--------------+---------+--------+



-------------------------------------------
Batch: 1
-------------------------------------------
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell_side|sell_product_id|sell_price|sell_time              |buy_side|buy_product_id|buy_price|buy_time               |
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell     |ETH-USD        |4048.37   |2021-11-28 18:04:58.761|buy     |ETH-USD       |4048.4   |2021-11-28 18:04:59.586|
|sell     |ETH-USD        |4048.37   |2021-11-28 18:05:01.2  |buy     |ETH-USD       |4048.38  |2021-11-28 18:05:01.297|
|sell     |ETH-USD        |4048.37   |2021-11-28 18:05:01.2  |buy     |ETH-USD       |4048.38  |2021-11-28 18:05:01.354|
|sell     |ETH-USD        |4048.37   |2021-11-28 18:05:01.2  |buy     |ETH-USD       |4048.48  |2021-11-28 18:05:02.163|
|sell     |ETH-USD        |4048.37   |2021-11-28 18:05:0

-------------------------------------------
Batch: 2
-------------------------------------------
+---------+---------------+----------+----------------------+--------+--------------+---------+-----------------------+
|sell_side|sell_product_id|sell_price|sell_time             |buy_side|buy_product_id|buy_price|buy_time               |
+---------+---------------+----------+----------------------+--------+--------------+---------+-----------------------+
|sell     |ETH-USD        |4048.2    |2021-11-28 18:05:03.14|buy     |ETH-USD       |4047.7   |2021-11-28 18:05:03.827|
+---------+---------------+----------+----------------------+--------+--------------+---------+-----------------------+



-------------------------------------------
Batch: 3
-------------------------------------------
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell_side|sell_product_id|sell_price|sell_time              |buy_side|buy_product_id|buy_price|buy_time               |
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell     |ETH-USD        |4046.66   |2021-11-28 18:05:05.262|buy     |ETH-USD       |4046.67  |2021-11-28 18:05:05.916|
|sell     |ETH-USD        |4046.66   |2021-11-28 18:05:06.295|buy     |ETH-USD       |4046.81  |2021-11-28 18:05:06.834|
|sell     |ETH-USD        |4046.53   |2021-11-28 18:05:06.295|buy     |ETH-USD       |4046.81  |2021-11-28 18:05:06.834|
|sell     |ETH-USD        |4046.66   |2021-11-28 18:05:06.295|buy     |ETH-USD       |4046.82  |2021-11-28 18:05:06.834|
|sell     |ETH-USD        |4046.53   |2021-11-28 18:05:0

...

-------------------------------------------
Batch: 13
-------------------------------------------
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell_side|sell_product_id|sell_price|sell_time              |buy_side|buy_product_id|buy_price|buy_time               |
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell     |ETH-USD        |4044.26   |2021-11-28 18:05:38.565|buy     |ETH-USD       |4043.84  |2021-11-28 18:05:38.834|
|sell     |ETH-USD        |4044.26   |2021-11-28 18:05:38.578|buy     |ETH-USD       |4043.84  |2021-11-28 18:05:38.834|
|sell     |ETH-USD        |4044.18   |2021-11-28 18:05:38.578|buy     |ETH-USD       |4043.84  |2021-11-28 18:05:38.834|
|sell     |ETH-USD        |4043.46   |2021-11-28 18:05:38.674|buy     |ETH-USD       |4043.84  |2021-11-28 18:05:38.834|
|sell     |ETH-USD        |4043.31   |2021-11-28 18:05:

-------------------------------------------
Batch: 14
-------------------------------------------
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell_side|sell_product_id|sell_price|sell_time              |buy_side|buy_product_id|buy_price|buy_time               |
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell     |ETH-USD        |4046.46   |2021-11-28 18:05:41.1  |buy     |ETH-USD       |4046.88  |2021-11-28 18:05:42.021|
|sell     |ETH-USD        |4047.49   |2021-11-28 18:05:44.352|buy     |ETH-USD       |4046.88  |2021-11-28 18:05:45.05 |
|sell     |ETH-USD        |4047.49   |2021-11-28 18:05:44.42 |buy     |ETH-USD       |4046.88  |2021-11-28 18:05:45.05 |
|sell     |ETH-USD        |4046.87   |2021-11-28 18:05:45.014|buy     |ETH-USD       |4046.88  |2021-11-28 18:05:45.05 |
|sell     |ETH-USD        |4047.49   |2021-11-28 18:05:

-------------------------------------------
Batch: 15
-------------------------------------------
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell_side|sell_product_id|sell_price|sell_time              |buy_side|buy_product_id|buy_price|buy_time               |
+---------+---------------+----------+-----------------------+--------+--------------+---------+-----------------------+
|sell     |ETH-USD        |4046.87   |2021-11-28 18:05:45.014|buy     |ETH-USD       |4046.08  |2021-11-28 18:05:45.434|
|sell     |ETH-USD        |4046.87   |2021-11-28 18:05:45.014|buy     |ETH-USD       |4046.59  |2021-11-28 18:05:45.756|
|sell     |ETH-USD        |4046.87   |2021-11-28 18:05:45.014|buy     |ETH-USD       |4046.59  |2021-11-28 18:05:45.843|
|sell     |ETH-USD        |4046.07   |2021-11-28 18:05:46.478|buy     |ETH-USD       |4046.59  |2021-11-28 18:05:46.674|
|sell     |ETH-USD        |4046.07   |2021-11-28 18:05:

-------------------------------------------
Batch: 16
-------------------------------------------
+---------+---------------+----------+----------------------+--------+--------------+---------+-----------------------+
|sell_side|sell_product_id|sell_price|sell_time             |buy_side|buy_product_id|buy_price|buy_time               |
+---------+---------------+----------+----------------------+--------+--------------+---------+-----------------------+
|sell     |ETH-USD        |4046.76   |2021-11-28 18:05:49.36|buy     |ETH-USD       |4046.55  |2021-11-28 18:05:50.173|
+---------+---------------+----------+----------------------+--------+--------------+---------+-----------------------+



21/11/28 18:05:58 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@124bdb47 is aborting.
21/11/28 18:05:58 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@124bdb47 aborted.
21/11/28 18:05:58 WARN Shell: Interrupted while joining on: Thread[Thread-163178,5,main]
java.lang.InterruptedException
	at java.base/java.lang.Object.wait(Native Method)
	at java.base/java.lang.Thread.join(Thread.java:1300)
	at java.base/java.lang.Thread.join(Thread.java:1375)
	at org.apache.hadoop.util.Shell.joinThread(Shell.java:1043)
	at org.apache.hadoop.util.Shell.runCommand(Shell.java:1003)
	at org.apache.hadoop.util.Shell.run(Shell.java:901)
	at org.apache.hadoop.util.Shell$ShellCommandExecutor.execute(Shell.java:1213)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1307)
	at org.apache.hadoop.util.Shell.execCommand(Shell.java:1289)
	at org.apache.hadoop.f